In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pylab as pl
import matplotlib.animation as animation
# This is how we can import src module from sibling module
import os, sys
nb_dir = os.path.split(os.getcwd())[0]
print nb_dir
if nb_dir not in sys.path: sys.path.append(nb_dir)
#######################
from src import wdata
from netCDF4 import Dataset
import seaborn as sns;
from src.wdata import Sat, Radar
from skimage.transform import resize
from sklearn.utils import shuffle
from random import random as random 

C:\Users\Sa\Desktop\capstone-weather


In [2]:
def div3(arr):
    a1 = arr[:,:52]
    a2 = arr[:,52:52*2]
    a3 = arr[:,52*2:156]
    return (a1, a2, a3)

def normBand(arr): # normalize and remap to [-1, 1]
    arr = arr - np.min(arr)
    arr = arr / np.max(arr)
    return arr

In [3]:
# todo: add code to automatically create directory structure where images will be saved
savedir = os.path.abspath("E:/weatherFall/run5")
radar_ = [i for mo in [7,8,9] for i in Radar.getMonth(2017,mo)]
bands, radar = [], []
sparsecount = 0
for tup in radar_:
    rad = tup[1][:,:156] # crop to 126x156
    # get corrosponding Satelite image
    ds2 = Sat.getSatFromRad(tup, 2)
    ds4 = Sat.getSatFromRad(tup, 4)
    ds6 = Sat.getSatFromRad(tup, 6)
    
    if ds2 and ds4 and ds6: 
        b2 = div3(resize(normBand(Sat.getAndClose(ds2)), (52,156)))
        b4 = div3(resize(normBand(Sat.getAndClose(ds4)), (52,156)))
        b6 = div3(resize(normBand(Sat.getAndClose(ds6)), (52,156)))
        rad1, rad2, rad3 = div3(rad)
        
        if Radar.isSparse(rad1, threshold=0.00175): 
            if random() < 0.05:               
                radar.append(rad1)
                bands.append(np.stack((b2[0],b4[0],b6[0]), axis=2))
                sparsecount += 1
        else:
            radar.append(rad1)
            bands.append(np.stack((b2[0],b4[0],b6[0]), axis=2))
        
        if Radar.isSparse(rad2, threshold=0.00175): 
            if random() < 0.05:    
                radar.append(rad2)
                bands.append(np.stack((b2[1],b4[1],b6[1]), axis=2))
                sparsecount += 1
        else:
            radar.append(rad2)
            bands.append(np.stack((b2[1],b4[1],b6[1]), axis=2))
            
        if Radar.isSparse(rad3, threshold=0.00175): 
            if random() < 0.05:    
                radar.append(rad3)
                bands.append(np.stack((b2[2],b4[2],b6[2]), axis=2))
                sparsecount += 1
        else:
            radar.append(rad3)
            bands.append(np.stack((b2[2],b4[2],b6[2]), axis=2))

bands, radar = shuffle(bands, radar)

d:\Anaconda2\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [4]:
size = len(bands)
print size
print len(bands), len(radar)
print 'Num of sparse images:', sparsecount

626
626 626
Num of sparse images: 44


In [ ]:
splitRatio = 0.80
def saveImgs():
    for i,d in enumerate(bands[:size]):
        plt.imsave(savedir+"/A/{:>03d}.png".format(i), d, cmap=None)
    print "done bands"
    for i,d in enumerate(radar[:size]):
        plt.imsave(savedir+"/B/{:>03d}.png".format(i), d*255, cmap='gray', origin='lower')
    print "done radar"
saveImgs()